<a href="https://colab.research.google.com/github/biplobsd/Google-Colab-CloudTorrent/blob/master/torrentTOmega_gdrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src='https://i.imgur.com/RhSYbDW.png' height="55" alt="Peerflix-server" />

In [0]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <br><center><img src='https://i.imgur.com/RhSYbDW.png' height="100" alt="netdata"/></center>
#@markdown <center><h3>☁️ Cloud Torrent: a self-hosted remote torrent client</h3></center><br>
import os, sys, subprocess; from IPython.display import clear_output

!pkill ssh
print("Loading ...")
if os.path.isfile("/usr/local/bin/cloud-torrent") == False:
  os.system('curl https://i.jpillora.com/cloud-torrent! | bash')
else:
  !pkill cloud-torrent
clear_output(wait=True)

#Forwarding public server by serveo.net
!nohup ssh -o StrictHostKeyChecking=no -R 80:localhost:4444 serveo.net > link.out &

#Opening cloud-torrent in background
!nohup cloud-torrent --port 4444 --title "Cloud Torrent" --log &
clear_output(wait=True)
print(os.popen('tail -n 2 link.out').read())

#MEGAcmd installing
if os.path.isfile("/usr/bin/mega-cmd") == False:
  print("Installing Mega ...")
  os.system('sudo apt-get -y update && sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https && sudo curl -sL -o/var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb && sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb && sudo ldconfig')
  print("Mega is installed.")
else:
  !pkill mega-cmd
#Google Drive CLI Client
if os.path.isfile("/usr/local/bin/gdrive") == False:
  print("Google Drive Cli Client installing ...")
  !wget -q -c -nc https://github.com/gdrive-org/gdrive/releases/download/2.1.0/gdrive-linux-x64
  !mv gdrive-linux-x64 /usr/local/bin/gdrive
  !sudo chmod +x /usr/local/bin/gdrive
  !rm -f /content/gdrive-linux-x64
  print("Google drive client installed")
else:
  !pkill gdrive
print("All Done.")

# <img src='https://camo.githubusercontent.com/95b0ba2a237ee846b3bb8f45954124eb9cba4ebc/68747470733a2f2f63646e2e6a7364656c6976722e6e65742f67682f617361706163682f70656572666c69782d736572766572406d61737465722f6170702f696d616765732f6c6f676f2e737667' height="25" alt="Peerflix-server" /> PEERFLIX-SERVER

In [0]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <br><center><img src='https://camo.githubusercontent.com/95b0ba2a237ee846b3bb8f45954124eb9cba4ebc/68747470733a2f2f63646e2e6a7364656c6976722e6e65742f67682f617361706163682f70656572666c69782d736572766572406d61737465722f6170702f696d616765732f6c6f676f2e737667' height="200" alt="netdata"/></center>
#@markdown <center><h3>PEERFLIX-SERVER<br />Streaming torrent client for node.js with web ui.</h3></center><br>
import sys, subprocess, string, os, urllib.request, json, time, getpass; from IPython.display import clear_output
!pkill autossh
!pkill nohup
!pkill peerflix-server

configPath = r"/root/.config/peerflix-server/config.json"
#CREATE THE CONFIG
data = r"""
{ 
   "path": "/content/peerflix",
   "connections": 400
}
""".split("\n")

# SAVE THE CONFIG
!mkdir -p /root/.config/peerflix-server/
open(configPath, 'w').close()

configFile = open(configPath, "a+")

for line in data:
  configFile.write(line + "\n")

configFile.close()

if os.path.isfile("/tools/node/bin/peerflix-server") == False:
  os.system('npm install -g peerflix-server')
else:
  !pkill peerflix-server
  
if os.path.isfile("/usr/bin/autossh") == False:
    os.system('sudo apt update && sudo apt install -y autossh')

if os.path.isfile("/usr/local/bin/ngrok") == False:
  !wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
  !unzip -qq -n ngrok-stable-linux-amd64.zip
  !mv ngrok /usr/local/bin/ngrok
  !rm -f /content/ngrok-stable-linux-amd64.zip
  clear_output(wait=True)
else:
  !pkill ngrok
  
print("Copy authtoken from https://dashboard.ngrok.com/auth")
authtoken = getpass.getpass()
get_ipython().system_raw('/usr/local/bin/ngrok authtoken $authtoken && /usr/local/bin/ngrok tcp 4445 &')

#Opening peerflix-server in background
!mkdir -p peerflix 
!PORT=4445 nohup peerflix-server &
clear_output(wait=True)
#Get public address and print connect command
time.sleep(10)
with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
  data = json.loads(response.read().decode())
  (host, port) = data['tunnels'][0]['public_url'][6:].split(':')
  print(f'Open this link in new tab http://{host}:{port}')

#MEGAcmd installing
if os.path.isfile("/usr/bin/mega-cmd") == False:
  print("Installing Mega ...")
  os.system('sudo apt-get -y update && sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https && sudo curl -sL -o/var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb && sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb && sudo ldconfig')
  print("Mega is installed.")
else:
  !pkill mega-cmd

#Google Drive CLI Client
if os.path.isfile("/usr/local/bin/gdrive") == False:
  print("Google Drive Cli Client installing ...")
  !wget -q -c -nc https://github.com/gdrive-org/gdrive/releases/download/2.1.0/gdrive-linux-x64
  !mv gdrive-linux-x64 /usr/local/bin/gdrive
  !sudo chmod +x /usr/local/bin/gdrive
  !rm -f /content/gdrive-linux-x64
  print("Google drive client installed")
else:
  !pkill gdrive
print("All Done.")

In [0]:

#@markdown <br><center><img src='https://www.iconsdb.com/icons/preview/red/warning-xxl.png' height="50" alt="netdata"/></center>
#@markdown <br><center><h1>CLEAN DOWNLODAS FOLDER</center>
#@markdown <br><center>PEERFLIX-SERVER don't delete removed torrent task file. So you do this manually after remove torrent task.
!touch /content/downloads/errorRemove
!cd downloads/ && sudo rm -r *
print("All Deleted on /content/peerflix")

<img src='http://pngimages.net/sites/default/files/zip-file-png-image-15464.png' height="30" alt="netdata"/>

#Extracte Zip

In [0]:
Cloud_torrent = True #@param {type:"boolean"}
Peerflix = False #@param {type:"boolean"}
#@markdown <br><center><img src='https://cdn0.iconfinder.com/data/icons/mattress-1/512/mattress-pad-bed-01-512.png' height="100" alt="netdata"/></center>
#@markdown <center><h2>Extracte Zip</h2></center><br>

if Cloud_torrent:
  !cd downloads/ && find . -name '*.zip' -exec sh -c 'unzip -d "`dirname \"{}\"`" "{}" && sudo rm "`dirname \"{}\"`" "{}"' ';'
else:
  print("Not enabled Cloud_torrent. Skipping....")
if Peerflix:  
  !cd peerflix/ && find . -name '*.zip' -exec sh -c 'unzip -d "`dirname \"{}\"`" "{}" && sudo rm "`dirname \"{}\"`" "{}"' ';'
else:
  print("Not enabled Peerflix. Skipping....")

<img src="https://upload.wikimedia.org/wikipedia/commons/a/ad/Logo_of_Google_Drive.svg" weight="100px" height="40px" />

#TRANSFERE to Google drive
<font color='red'>I recommend to use native drive mount and use drag and drop to transfer data to google drive.</font> [Read more.](https://colab.research.google.com/drive/1srw_HFWQ2SMgmWIawucXfusGzrj1_U0q#scrollTo=UADu4vFNkjX1)

In [0]:
#@markdown <h3>⬅️ Click Here to</h3>

#@markdown #INPUT YOUR GOOGLE DRIVE authtoken
#@markdown #### (SOMETIMES WRITING ISSUES. SO RUN AGAIN)

!gdrive about


In [0]:
Cloud_torrent = True #@param {type:"boolean"}
Peerflix = False #@param {type:"boolean"}
#UPLOAD TO GOOGLE DRIVE
#@markdown <br><center><img src='https://support.questetra.com/wp-content/uploads/2018/04/googledrive-simpleupload-fi-en.png' height="200" alt="netdata"/></center>
#@markdown <center><h2>TRANSFERE NOW</h2></center><br>

if Cloud_torrent:
  !gdrive upload --recursive /content/downloads
else:
  print("Not enabled Cloud_torrent. Skipping....")
if Peerflix:
  !gdrive upload --recursive /content/peerflix
else:
  print("Not enabled Peerflix. Skipping....")


![alt text](https://mega.nz/favicon.ico?v=3) 
#TRANSFERE to MEGA

---





In [0]:
#@markdown <center><h2>INPUT YOUR MEGA ID</h2></center><br>
#@markdown <center>(If you see Fetching nodes 100% on stuck, rerun)</center><br>

import os
import contextlib
import subprocess
import time

#INPUT YOUR MEGA ID

USERNAME = "" #@param {type:"string"}
PASSWORD = "" #@param {type:"string"}
# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']
def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out

def example():
    cmd = ['mega-login', USERNAME, PASSWORD]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        clear_output(wait=True)
        print(line)
example()
!mega-whoami


In [0]:
Cloud_torrent = True #@param {type:"boolean"}
Peerflix = False #@param {type:"boolean"}
#@markdown <br><center><img src='https://www.impactcomputing.co.uk/uploads/files/services/managed-office-365.svg' height="100" alt="netdata"/></center>
#@markdown <center><h2>TRANSFERE NOW</h2></center><br>
#TRANSFERE NOW

import contextlib
import subprocess
import time


# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']
def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out

def example():
    if Cloud_torrent:
      cmd = ['mega-put', 'downloads', '/colab']
    if Peerflix:
      cmd = ['mega-put', 'peerflix', '/colab']
      
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        clear_output(wait=True)
        print(line)

example()